## Experiments with rank of core tensors and shape of input tensor on the MNIST dataset

In [0]:
# Basic
import numpy as np

# Keras Model
from keras.layers import Input, Dense
from keras.models import Model
from keras.regularizers import l2
from keras.optimizers import *

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
os.chdir('gdrive/My Drive/NLA PROJECT')

In [0]:
# TT Layer
from TTLayer import *

# Data
from keras.datasets import mnist

# Others
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score

np.random.seed(11111986)

In [0]:
# Load the MNIST data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.astype('float32')
y_train = y_train.astype('int32')
X_test = X_test.astype('float32')
y_test = y_test.astype('int32')

Y_train = to_categorical(y_train, 10)
Y_test = to_categorical(y_test, 10)

In [0]:
# Put 2 arrays on the border of the images to form a 32x32 shape
N = X_train.shape[0]
left0 = np.zeros((N, 2, 28))
right0 = np.zeros((N, 2, 28))
upper0 = np.zeros((N, 32, 2))
lower0 = np.zeros((N, 32, 2))

X_train = np.concatenate([left0, X_train, right0], axis=1)
X_train = np.concatenate([upper0, X_train, lower0], axis=2)

In [0]:
N = X_test.shape[0]
left0 = np.zeros((N, 2, 28))
right0 = np.zeros((N, 2, 28))
upper0 = np.zeros((N, 32, 2))
lower0 = np.zeros((N, 32, 2))

X_test = np.concatenate([left0, X_test, right0], axis=1)
X_test = np.concatenate([upper0, X_test, lower0], axis=2)

In [0]:
X_train /= 255.
X_test /= 255.

X_train = X_train[:, None, :, :]
X_test = X_test[:, None, :, :]

In [0]:
if False:  # if apply the imagegenerator
    valid_size = int(0.2*X_train.shape[0])
    valid_inds = np.random.choice(range(X_train.shape[0]), valid_size, False)
    X_valid = X_train[valid_inds]
    Y_valid = Y_train[valid_inds]

    tr_inds = np.setdiff1d(np.arange(X_train.shape[0]), valid_inds)
    X_train = X_train[tr_inds]
    Y_train = Y_train[tr_inds]

    train_gen = ImageDataGenerator(
        featurewise_center=True,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=30,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images
    train_gen.fit(X_train)

    valid_gen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        rotation_range=0,
        width_shift_range=0,
        height_shift_range=0,
        horizontal_flip=False,
        vertical_flip=False
    )
    valid_gen.fit(X_valid)


In [0]:
# Define the model
input = Input(shape=(1, 32, 32,))
h1 = TT_Layer(tt_input_shape=[4, 8, 8, 4], tt_output_shape=[8, 8, 8, 8], tt_ranks=[1, 1, 1, 1, 1],
              use_bias=True, activation='relu', kernel_regularizer=l2(5e-4), debug=False)(input)
# Alternatively, try a dense layer:
# h1 = Dense(output_dim=32*32, activation='relu', kernel_regularizer=l2(5e-4))(input)
output = Dense(output_dim=10, activation='softmax', kernel_regularizer=l2(1e-3))(h1)

model = Model(input=input, output=output)
model.compile(optimizer=Adam(1e-3), loss='categorical_crossentropy', metrics=['accuracy'])

Compression factor = 192 / 4194304 = 4.57763671875e-05


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", kernel_regularizer=<keras.reg..., units=10)`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


In [0]:
# Train the model
# either the old fashion:
model.fit(x=X_train, y=Y_train, verbose=2, epochs=50, batch_size=128,
          validation_split=0.2)

# or with ImageDataGenerator
# model.fit_generator(train_gen.flow(X_train, Y_train, batch_size=128),
#                     samples_per_epoch=len(X_train), nb_epoch=50, verbose=2,
#                     validation_data=valid_gen.flow(X_valid, Y_valid),
#                     nb_val_samples=X_valid.shape[0])

Train on 48000 samples, validate on 12000 samples
Epoch 1/50
 - 22s - loss: 0.6947 - acc: 0.8275 - val_loss: 0.3366 - val_acc: 0.9276
Epoch 2/50
 - 23s - loss: 0.3012 - acc: 0.9357 - val_loss: 0.2649 - val_acc: 0.9452
Epoch 3/50
 - 23s - loss: 0.2468 - acc: 0.9481 - val_loss: 0.2271 - val_acc: 0.9539
Epoch 4/50
 - 24s - loss: 0.2195 - acc: 0.9555 - val_loss: 0.2212 - val_acc: 0.9551
Epoch 5/50
 - 24s - loss: 0.2003 - acc: 0.9597 - val_loss: 0.2018 - val_acc: 0.9590
Epoch 6/50
 - 24s - loss: 0.1889 - acc: 0.9635 - val_loss: 0.1938 - val_acc: 0.9599
Epoch 7/50
 - 24s - loss: 0.1783 - acc: 0.9657 - val_loss: 0.1938 - val_acc: 0.9617
Epoch 8/50
 - 24s - loss: 0.1716 - acc: 0.9665 - val_loss: 0.1983 - val_acc: 0.9608
Epoch 9/50
 - 24s - loss: 0.1642 - acc: 0.9698 - val_loss: 0.1796 - val_acc: 0.9645
Epoch 10/50
 - 24s - loss: 0.1595 - acc: 0.9711 - val_loss: 0.1861 - val_acc: 0.9641
Epoch 11/50
 - 25s - loss: 0.1531 - acc: 0.9719 - val_loss: 0.1817 - val_acc: 0.9632
Epoch 12/50
 - 25s - los

In [0]:
# Fitted values: AUROC/AUPRC/ACC
Y_hat = model.predict(x=X_train)
print (roc_auc_score(Y_train, Y_hat))
print (average_precision_score(Y_train, Y_hat))
print (accuracy_score(Y_train, np.round(Y_hat)))


# Predicted values:
Y_pred = model.predict(x=X_test)
print (roc_auc_score(Y_test, Y_pred))
print (average_precision_score(Y_test, Y_pred))
print (accuracy_score(Y_test, np.round(Y_pred)))

In [0]:
# Without the TT Layer:

X_train = X_train.reshape((X_train.shape[0], 32*32))
X_test = X_test.reshape((X_test.shape[0], 32*32))

input = Input(shape=(32*32,))
h1 = Dense(output_dim=32*32, activation='relu', kernel_regularizer=l2(5e-4))(input)
output = Dense(output_dim=10, activation='softmax', kernel_regularizer=l2(1e-3))(h1)
model = Model(input=input, output=output)
model.compile(optimizer=Adam(1e-3), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x=X_train, y=Y_train, verbose=2, nb_epoch=50, batch_size=128,
          validation_split=0.2)

C:\Users\alkim_000\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", kernel_regularizer=<keras.reg..., units=1024)`
  import sys
C:\Users\alkim_000\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", kernel_regularizer=<keras.reg..., units=10)`
  
C:\Users\alkim_000\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  if __name__ == '__main__':
C:\Users\alkim_000\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 48000 samples, validate on 12000 samples
Epoch 1/50
 - 31s - loss: 0.4806 - acc: 0.9214 - val_loss: 0.3140 - val_acc: 0.9539
Epoch 2/50
 - 28s - loss: 0.2800 - acc: 0.9595 - val_loss: 0.2562 - val_acc: 0.9644
Epoch 3/50
 - 35s - loss: 0.2455 - acc: 0.9664 - val_loss: 0.2490 - val_acc: 0.9642
Epoch 4/50
 - 35s - loss: 0.2318 - acc: 0.9700 - val_loss: 0.2369 - val_acc: 0.9664
Epoch 5/50
 - 33s - loss: 0.2256 - acc: 0.9706 - val_loss: 0.2587 - val_acc: 0.9614
Epoch 6/50
 - 33s - loss: 0.2228 - acc: 0.9724 - val_loss: 0.2288 - val_acc: 0.9706
Epoch 7/50
 - 31s - loss: 0.2194 - acc: 0.9728 - val_loss: 0.2366 - val_acc: 0.9675
Epoch 8/50
 - 28s - loss: 0.2204 - acc: 0.9725 - val_loss: 0.2331 - val_acc: 0.9667
Epoch 9/50
 - 30s - loss: 0.2180 - acc: 0.9732 - val_loss: 0.2205 - val_acc: 0.9719
Epoch 10/50
 - 31s - loss: 0.2145 - acc: 0.9744 - val_loss: 0.2277 - val_acc: 0.9694
Epoch 11/50
 - 30s - loss: 0.2135 - acc: 0.9744 - val_loss: 0.2244 - val_acc: 0.9697
Epoch 12/50
 - 31s - los

In [0]:
# Fitted values: AUROC/AUPRC/ACC
Y_hat = model.predict(x=X_train)
print (roc_auc_score(Y_train, Y_hat))
print (average_precision_score(Y_train, Y_hat))
print (accuracy_score(Y_train, np.round(Y_hat)))


# Predicted values:
Y_pred = model.predict(x=X_test)
print (roc_auc_score(Y_test, Y_pred))
print (average_precision_score(Y_test, Y_pred))
print (accuracy_score(Y_test, np.round(Y_pred)))

0.9996275351890462
0.9975178002671339
0.97405
0.9995422723428502
0.9967650523687439
0.9675
